In [1]:
from arc import ARC
from arc.common import save_yaml_file
from rmgpy.chemkin import load_chemkin_file
from rmgpy.reaction import Reaction
from rmgpy.chemkin import load_species_dictionary
import os
import yaml

/home/nelly/Code/RMG-Py/rmgpy/rmg/reactors.py:52: RuntimeWarning: Unable to import Julia dependencies, original error: No module named 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


In [37]:
run_dir = 'xmr_2045_p'
chemkin_path = os.path.join(os.getcwd(),run_dir,'chem_annotated.inp')
dict_path = os.path.join(os.getcwd(),run_dir,'species_dictionary.txt')
output_path = os.path.join(os.getcwd(),run_dir,'sa_summary_input.yml')
output_path_2 = os.path.join(os.getcwd(),run_dir,'GA_input.yml')
summary_yaml_path = os.path.join(os.getcwd(),run_dir,'summary.yml')

#get the pool combined yaml file ( all specis in pool)
scripts_dir = os.path.split(os.path.abspath(os.getcwd()))[0]
pool_yaml_path = os.path.join(scripts_dir,'POOL','combined_pool.yml')

In [47]:
#Do not change the Chemkin file from RMG, just copy paste it
species, reactions = load_chemkin_file(path=chemkin_path,
                                       dictionary_path=dict_path,
                                       check_duplicates=False,
                                       use_chemkin_names=True,
                                      )

label_adj_dict = load_species_dictionary(dict_path)  

317


In [4]:
def fix_stoich(reactants):
    i = 0
    while i < len(reactants):
        if reactants[i].isdigit():  
            multiplier = int(reactants[i]) 
            if i + 1 < len(reactants):  
                reactant = reactants[i + 1]
                repeated_reactant = [reactant] * (multiplier-1) #since one occurance already exists
                reactants[i + 1:i + 1] = repeated_reactant
            del reactants[i]
        else:
            i += 1
    return(reactants)

In [5]:
from rmgpy.reaction import Reaction
from rmgpy.species import Species
def get_rxn(rxn_label, label_adj_dict):
    try:
        if ' <=> ' in rxn_label:
            label_splits = rxn_label.split(' <=> ')
        elif ' => ' in rxn_label:
            label_splits = rxn_label.split(' => ')
    except:
        raise ValueError("Invalid reaction format: must contain '=> ' or '<=> '")
    reactants = label_splits[0].split(' + ')
    reactants = [i.split(' ') for i in reactants]
    reactants = [value for sublist in reactants for value in sublist]
    reactants = fix_stoich(reactants)
    products = label_splits[1].split(' + ')
    products = [i.split(' ') for i in products]
    products = [value for sublist in products for value in sublist]
    products = fix_stoich(products)

    reactants = {label: label_adj_dict[label] for label in reactants if label != '(+M)'}
    products = {label: label_adj_dict[label] for label in products if label != '(+M)'}
    
    #merge dicts uniquely
    species_dict = reactants.copy()  # Make a copy of original dict
    species_dict.update(products)

    return species_dict

In [6]:
#Read summary yaml file of SA PFR (Lag method) rxns 
with open(summary_yaml_path, 'r') as file:
    summary_yaml_data = yaml.safe_load(file)

summary_species = {}
for specie in summary_yaml_data.keys():
    for T in summary_yaml_data[specie].keys():
        for i in summary_yaml_data[specie][T].keys():
            summary_species.update(get_rxn(summary_yaml_data[specie][T][i]['reaction'], label_adj_dict))


In [7]:
#Read pool yaml file 
with open(pool_yaml_path, 'r') as file:
    pool_yaml_data = yaml.safe_load(file)

In [8]:
def extract_filename(file_path):
    base_filename = os.path.basename(file_path)
    filename_without_extension = os.path.splitext(base_filename)[0]
    return filename_without_extension

#store adj_lists of the pool in a list
pool_species = {}
for key,value in pool_yaml_data.items():
    label = extract_filename(key)
    adj_list = value['adjacency_list']
    pool_species[label] = Species().from_adjacency_list(adj_list)


In [ ]:
#Filter out species from summary SA which does appear in POOL

summary_final_list = []
for summary_specie in summary_species.values():
    flag = False #not in pool
    for pool_specie in pool_species.values():
        if summary_specie.is_isomorphic(pool_specie, strict = True):
            flag = True
    if not flag:
        summary_final_list.append(summary_specie)


In [ ]:
#Also filter out species from summary SA which were not thermo estimated by group additiity

for spc in species: #iterate over all species from chemkin
#check whether spc appears in sa species
    for sa_spc in summary_final_list:
        if spc.is_isomorphic(sa_spc, strict = True):
        #take only species that were estimated by group additiity
            if not ('group(' in spc.thermo.comment or 'radical(' in spc.thermo.comment):
                summary_final_list.remove(sa_spc)


In [29]:
# #Convert RMG species into ARC species

# from arc.species import ARCSpecies

# arc_summary_final_list = []
# for specie in summary_final_list:
#     arc_summary_final_list.append(ARCSpecies(rmg_species=specie))

In [36]:
content = list()

for spc in summary_final_list:
    content.append({'label': spc.label, 'adjlist': spc.molecule[0].to_adjacency_list()})

#ess_settings = {'gaussian': 'local'}
job_types = {'conformers': True,
             'opt': True,
             'fine_grid': True,
             'freq': True,
             'bde': False,
             'sp': True,
             'rotors': True,
             'irc': False,
            }

input_sa_yml_dict = {}
input_sa_yml_dict['project'] = 'xmr_2045_p_sa_summary'
input_sa_yml_dict['job_memory'] = 50
input_sa_yml_dict['job_types'] = job_types
input_sa_yml_dict['level_of_theory'] = 'CBS-QB3'
input_sa_yml_dict['allow_nonisomorphic_2d'] = True
input_sa_yml_dict['ts_adapters'] = ['AutoTST', 'heuristics']
input_sa_yml_dict['species'] = content

save_yaml_file(path=output_path,
               content=input_sa_yml_dict,
              )

In [ ]:
#Filter out species from species list (Chemkin) does appear in POOL and in SA species
#Add only those that were estimated by GA

spc_final_list = []
for spc in species:
    flag = False #not in pool
    for pool_spc in pool_species.values():
        if spc.is_isomorphic(pool_spc, strict = True):
            flag = True
    for sa_spc in summary_final_list:
        if spc.is_isomorphic(sa_spc, strict = True):
            flag = True
    if not flag and 'group(' in spc.thermo.comment or 'radical(' in spc.thermo.comment:
        spc_final_list.append(spc)


In [49]:
content = list()

for spc in spc_final_list:
    content.append({'label': spc.label+"(" + str(spc.index) + ")", 'adjlist': spc.molecule[0].to_adjacency_list()})

#ess_settings = {'gaussian': 'local'}
job_types = {'conformers': True,
             'opt': True,
             'fine_grid': True,
             'freq': True,
             'bde': False,
             'sp': True,
             'rotors': True,
             'irc': False,
            }

input_sa_yml_dict = {}
input_sa_yml_dict['project'] = 'xmr_2045_p_NPS_summary'
input_sa_yml_dict['job_memory'] = 50
input_sa_yml_dict['job_types'] = job_types
input_sa_yml_dict['level_of_theory'] = 'CBS-QB3'
input_sa_yml_dict['allow_nonisomorphic_2d'] = True
input_sa_yml_dict['ts_adapters'] = ['AutoTST', 'heuristics']
input_sa_yml_dict['species'] = content

save_yaml_file(path=output_path_2,
               content=input_sa_yml_dict,
              )

In [50]:
print(len(content))

207
